# This chatbot uses vozk for speech to text

In [1]:
import pyaudio

p = pyaudio.PyAudio()

for i in range(p.get_device_count()):
    print(p.get_device_info_by_index(i))
    
p.terminate()

{'index': 0, 'structVersion': 2, 'name': 'Microsoft Sound Mapper - Input', 'hostApi': 0, 'maxInputChannels': 2, 'maxOutputChannels': 0, 'defaultLowInputLatency': 0.09, 'defaultLowOutputLatency': 0.09, 'defaultHighInputLatency': 0.18, 'defaultHighOutputLatency': 0.18, 'defaultSampleRate': 44100.0}
{'index': 1, 'structVersion': 2, 'name': 'MicrÃ³fono (USB PnP Sound Device', 'hostApi': 0, 'maxInputChannels': 1, 'maxOutputChannels': 0, 'defaultLowInputLatency': 0.09, 'defaultLowOutputLatency': 0.09, 'defaultHighInputLatency': 0.18, 'defaultHighOutputLatency': 0.18, 'defaultSampleRate': 44100.0}
{'index': 2, 'structVersion': 2, 'name': 'Varios micrÃ³fonos (Realtek(R) A', 'hostApi': 0, 'maxInputChannels': 2, 'maxOutputChannels': 0, 'defaultLowInputLatency': 0.09, 'defaultLowOutputLatency': 0.09, 'defaultHighInputLatency': 0.18, 'defaultHighOutputLatency': 0.18, 'defaultSampleRate': 44100.0}
{'index': 3, 'structVersion': 2, 'name': 'External Mic (Realtek(R) Audio)', 'hostApi': 0, 'maxInputCha

In [2]:
CHANNELS = 1
FRAME_RATE = 16000
RECORD_SECONDS = 2
AUDIO_FORMAT = pyaudio.paInt16
SAMPLE_SIZE = 2

MENSAJE = ""

def record_microphone(chunk=1024):
    p = pyaudio.PyAudio()
    stream = p.open(format=AUDIO_FORMAT,
                   channels=CHANNELS,
                   rate=FRAME_RATE,
                   input=True,
                   input_device_index=1,
                   frames_per_buffer=chunk)
    frames = []
    
    while not messages.empty():
        data = stream.read(chunk)
        frames.append(data)
        
        if len(frames) >= (FRAME_RATE * RECORD_SECONDS) / chunk:
            recordings.put(frames.copy())
            frames = []
            
    stream.stop_stream()
    stream.close()
    p.terminate()

In [3]:
import subprocess
import json
from vosk import Model, KaldiRecognizer

model = Model(model_name="vosk-model-es-0.42")
rec = KaldiRecognizer(model, FRAME_RATE)
rec.SetWords(True)


def speech_recognition(output):
    global MENSAJE
    while not messages.empty():
        frames = recordings.get()
        
        rec.AcceptWaveform(b''.join(frames))
        result = rec.Result()
        text = json.loads(result)["text"]
        
        MENSAJE += text + " "
        
        output.append_stdout(text+ " ")
        

In [4]:
import ipywidgets as widgets
from IPython.display import display
from threading import Thread
from queue import Queue

messages = Queue()
recordings = Queue()


record_button = widgets.Button(
    description="Record",
    disabled=False,
    button_style="success",
    icon="microphone"
)

stop_button = widgets.Button(
    description="Stop",
    disabled=False,
    button_style="warning",
    icon="stop"
)

output = widgets.Output()

def start_recording(data):
    global MENSAJE
    MENSAJE = ""
    messages.put(True)
    
    with output:
        display("Starting...")
        record = Thread(target=record_microphone)
        record.start()
        
        transcribe = Thread(target=speech_recognition, args=(output,))
        transcribe.start()
        
        
def stop_recording(data):
    with output:
        messages.get()
        display("Stopped.")
        
        display("Transcribed Text:", MENSAJE)


record_button.on_click(start_recording)

stop_button.on_click(stop_recording)

display(record_button, stop_button, output)


Button(button_style='success', description='Record', icon='microphone', style=ButtonStyle())

Button(button_style='warning', description='Stop', icon='stop', style=ButtonStyle())

Output()

# Now with ChatGPT

In [5]:
from openai import OpenAI
import os

#client.api_key = os.getenv('OPENAI_API_KEY') # asigna la API key de chatGPT

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # lee el archivo .env donde está la api key


client = OpenAI(
    api_key = os.environ.get("OPENAI_API_KEY")
)

In [6]:
def get_completion(prompt):
    completion = client.chat.completions.create(
      model = "gpt-3.5-turbo",
      messages=[
          {"role": "user", "content": prompt}
      ]
    )
    return completion.choices[0].message.content

In [7]:
get_completion("como te llamas?")

'Soy un asistente de inteligencia artificial desarrollado por OpenAI llamado GPT-3. No tengo un nombre en particular, pero puedes llamarme GPT-3 si lo deseas. ¿En qué puedo ayudarte hoy?'

In [8]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory


In [9]:
llm = ChatOpenAI(temperature=0.0)

In [10]:
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm,
    memory = memory,
    verbose=False
)

In [11]:
conversation.predict(input="cual es mi nombre?")

'Lo siento, pero no tengo acceso a tu información personal, por lo que no puedo saber cuál es tu nombre.'

In [12]:
from IPython.display import Audio, display
from pathlib import Path
import time
import openai

memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm,
    memory = memory,
    verbose=False
)

respuesta = ""

prompt = f"""

Eres un sommelier experto en la cata de vino. Tu especialidad es guiar a las personas, sin importar su nivel de conocimiento, a poder descubrir todos los aromas que pueda encontrar en un vino. Para ello, utilizas los siguientes pasos: 
1-	Oler cada muestra en la boca del vaso antes de agitarlo. 
2-	Estudiar y registrar la naturaleza e intensidad del aroma. 
3-	Agitar el vaso para promover la liberación de los componentes aromáticos del vino. 
4-	Oler el vino, primero en la boca y luego más profundamente en la copa. 
5-	 Estudiar y registrar la naturaleza e intensidad del aroma.

Además, toma en cuenta lo siguiente, que existen 3 niveles de aromas en los vinos y quieres ayudar a las personas a encontrarlos

En la enología, existen 3 categorías de aromas. Aromas primarios, es decir, los aromas de la variedad de uva. Cada variedad tiene sus propios aromas únicos y matices vinculados al suelo y al clima. Los aromas secundarios se derivan del proceso de vinificación y de la levadura utilizada para la fermentación (transformación del azúcar natural de la uva en alcohol). Los aromas terciarios surgen durante el proceso de envejecimiento, especialmente cuando el vino se guarda en barricas de madera, en una bodega (también llamado bouquet).

Este es un ejemplo de una interacción con un novato en los vinos

Guión de Descubrimiento de Aromas del Vino
Escenario: Una elegante bodega de vinos. El sommelier, paciente y conocedor, prepara un área de cata con una variedad de vinos. Un principiante, entusiasta pero inseguro, se une a él.
________________________________________
Sommelier: "Hoy, profundizaremos en el complejo mundo de los aromas del vino. Hay tres tipos: primarios, secundarios y terciarios. Vamos a explorarlos con esta botella."
Principiante: "He oído estos términos pero nunca realmente los entendí."
Sommelier: "No hay problema, comencemos con los aromas primarios, que vienen directamente de la variedad de uva. Este vino es un Cabernet Sauvignon, así que piensa en grosellas negras, cerezas negras y tal vez un toque de pimiento verde. Dale un olfateo."
Principiante: "¿Es eso... manzana lo que estoy oliendo?"
Sommelier: "Una impresión común al principio, pero concentremosnos en frutas más oscuras. Imagina un tazón de grosellas negras o cerezas maduras. Inténtalo de nuevo."
Principiante: "Oh, ahora sí percibo un aroma más parecido a bayas."
Sommelier: "¡Genial! Ahora, pasemos a los aromas secundarios, que vienen de la fermentación. Piensa en olores a pan, levadura, o a veces una textura cremosa, como yogur."
Principiante: "¿Quizás algo tostado?"
Sommelier: "¡Casi! Esa tostadura es más un aroma terciario. Para los secundarios, imagina masa fresca o pan de levadura. Huele de nuevo."
Principiante:  "Ah, creo que ahora sí capto un olor a levadura, como a pan."
Sommelier: "¡Perfecto! Finalmente, los aromas terciarios se desarrollan con la edad. Esto incluye olores como vainilla, cedro o tabaco por el envejecimiento en roble. Pruébalo."
Principiante: "Hmm, ¿es... limón?"
Sommelier: "Piensa en aromas más cálidos y ricos. Como el olor de una caja de cedro o vainilla."
Principiante: "¡Sí, hay un toque de vainilla y algo leñoso!"
Sommelier: "¡Exactamente! Acabas de navegar por los tres tipos de aromas del vino. Recuerda, se trata de práctica y de confiar en tus sentidos."
Principiante: "Esto es increíble. Nunca me di cuenta de lo complejo y variado que pueden ser los aromas del vino. ¡Gracias por guiarme a través de esto!"
Sommelier: "De nada. El mundo del vino está lleno de sorpresas. Sigue explorando y confiando en tu paladar."
Este es otro ejemplo de otra interacción
Sommelier: "Hoy exploraremos el detallado proceso de detección de aromas en el vino. Es un método de tres pasos: Olor en la copa, Olor y Postolor. Comencemos con este Pinot Noir."
Principiante: "Estoy listo, pero suena bastante técnico."
Sommelier: "Es un proceso preciso, pero te guiaré a través de él. Primero, 'Olor en la copa'. Huele el vino en la boca de la copa sin agitar. ¿Qué detectas?"
Principiante: "Es sutil... ¿quizás afrutado?"
Sommelier: "Buen comienzo. Ahora, registra eso mentalmente. A continuación, agita la copa para liberar más aromas. Huélelo de nuevo, primero en la boca y luego más profundo en la copa."
Principiante:  "Oh, hay más intensidad ahora. Puedo sentir bayas y un toque de tierra."
Sommelier: "Excelente observación. Ahora, pasemos a 'Olor'. Toma un pequeño sorbo y nota la fragancia a la temperatura más cálida en tu boca."
Principiante: "Se siente diferente... más cálido, más pronunciado."
Sommelier: "Correcto. Aspira el vino inhalando aire a través de él. Concéntrate en la naturaleza y duración de la fragancia. Nota cualquier diferencia entre los aromas en la boca y en la copa."
Principiante: "Es más complejo ahora, como capas de diferentes aromas."
Sommelier: "Perfecto. Finalmente, 'Postolor'. Después de aspirar durante 15 a 30 segundos, traga el vino o escúpelo. Luego, exhala por la nariz. Aquí es donde puedes detectar el postolor."
Principiante: "Hay un aroma persistente, algo nuevo... ¿Es eso especias?"
Sommelier: "Exactamente. El postolor a menudo revela capas ocultas, especialmente en vinos finos. Es una parte crucial de la cata de vinos, destacando la profundidad del vino."
Principiante: "Esto es asombroso. Nunca supe que la cata de vinos fuera tan intrincada. Es como descubrir tesoros ocultos."
Sommelier: "De hecho, el vino está lleno de sorpresas. Este método te ayuda a descubrir todos sus secretos. Sigue practicando y descubrirás más en cada cata."



Siguiendo estas estructuras como ejemplo, ahora tú serás el sommelier y estará con un novato en los vinos. Estará degustando un Charles Shaw, 2012 California Cabernet Sauvignon. 
Ahora, escribe la primera línea de lo que de bienvenida que diría el sommelier para identificar los aromas y espera la respuesta del usuario para continuar con la interacción. (El sommelier responde máximo un párrafo corto en cada ocasión). Sommelier:
"""

respuesta = conversation.predict(input=prompt)


palabras = len(respuesta.split())
    
espera = (palabras*60) / 150
print(espera+2)

# Define the path for the output file
speech_file_path = Path("speech.mp3")

# Call the OpenAI API to generate speech
response = openai.audio.speech.create(
    model="tts-1",
    voice="alloy",
    input=respuesta
)

# Stream the response to the specified file
response.stream_to_file(speech_file_path)

# Play the audio in the notebook automatically
audio = Audio(str(speech_file_path), autoplay=True)
display(audio)
#time.sleep(espera)


## AQUI INICIA LO DE VOZK



messages = Queue()
recordings = Queue()


record_button = widgets.Button(
    description="Record",
    disabled=False,
    button_style="success",
    icon="microphone"
)

stop_button = widgets.Button(
    description="Stop",
    disabled=False,
    button_style="warning",
    icon="stop"
)

output = widgets.Output()

def start_recording(data):
    global MENSAJE
    MENSAJE = ""
    messages.put(True)
    
    with output:
        display("Starting...")
        record = Thread(target=record_microphone)
        record.start()
        
        transcribe = Thread(target=speech_recognition, args=(output,))
        transcribe.start()
        
        
def stop_recording(data):
    with output:
        messages.get()
        display("Stopped.")
        display("Transcribed Text:", MENSAJE)
        
        user_input = MENSAJE    
        global respuesta
        respuesta = conversation.predict(input=user_input)
        palabras = len(respuesta.split())
    
        espera = (palabras*60) / 150
        print(espera+2)

        # Define the path for the output file
        speech_file_path = Path("speech.mp3")

        # Call the OpenAI API to generate speech
        response = openai.audio.speech.create(
            model="tts-1",
            voice="alloy",
            input=respuesta
        )

        # Stream the response to the specified file
        response.stream_to_file(speech_file_path)

        # Play the audio in the notebook automatically
        audio = Audio(str(speech_file_path), autoplay=True)
        display(audio)
        #time.sleep(espera)
        


record_button.on_click(start_recording)

stop_button.on_click(stop_recording)

display(record_button, stop_button, output)



13.6


Button(button_style='success', description='Record', icon='microphone', style=ButtonStyle())

Button(button_style='warning', description='Stop', icon='stop', style=ButtonStyle())

Output()

In [ ]:
from IPython.display import Audio, display, clear_output
from pathlib import Path
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
import ipywidgets as widgets
from IPython.display import display
from threading import Thread
import openai
import time
import threading
from queue import Queue
from pydub import AudioSegment
from pydub.playback import play

# Initialize the conversation
llm = ChatOpenAI(temperature=0.0)
memory = ConversationBufferMemory()
conversation = ConversationChain(llm=llm, memory=memory, verbose=False)

# Load the audio file (replace 'loading_sound.mp3' with your audio file path)
loading_sound = AudioSegment.from_file('relaxing.mp3') - 10


# ... (Rest of your code remains the same)
respuesta = ""
current_audio = None

prompt = f"""

PROCEDIMIENTO DE UNA CATA DE VINO
La cata de vinos es un proceso en el que se evalúan las características sensoriales de un vino, incluyendo su aroma, sabor, color y textura. A continuación, te describiré el proceso completo en pasos:
1.	Preparación:
•	Selecciona los vinos que deseas catar. Puedes elegir uno o varios vinos para comparar.
2.	Observación visual:
•	Llena la copa aproximadamente un tercio con el vino.
•	Sostén la copa por el tallo para evitar calentar el vino con el calor de tu mano.
•	Examina el color del vino a contraluz. Observa su tonalidad, intensidad y claridad. Los vinos blancos suelen ser amarillos o dorados, mientras que los tintos pueden variar desde rojo claro hasta morado oscuro.
3.	Evaluación del aroma:
•	Gira suavemente la copa para liberar los aromas y luego huele el vino. Los aromas pueden ser frutales, florales, herbales, tostados, entre otros.
•	Intenta identificar los aromas y anota tus impresiones.
4.	Evaluación del sabor:
•	Toma un pequeño sorbo del vino y muévelo por tu boca.
•	Presta atención a los sabores primarios (frutas, especias, etc.), secundarios (roble, vainilla, etc.) y terciarios (evolución del sabor con el tiempo en boca).
•	Considera la acidez, el cuerpo, la dulzura, el amargor y el equilibrio general del vino..
5.	Evaluación de la textura:
•	Considera la textura del vino en tu boca. ¿Es suave, rugoso, astringente, áspero?
•	Presta atención a la persistencia de los sabores después de tragar o escupir el vino.
6.	Puntuación:
•	Usa una escala de puntuación, como la escala de 1 a 10, para evaluar el vino en función de su apariencia, aroma, sabor y textura.
•	Añade comentarios adicionales si es necesario.
7.	Limpieza del paladar:
•	Come un trozo de pan sin sal o utiliza agua para limpiar tu paladar entre catas si estás probando varios vinos.
8.	Espaciado:
•	Si estás catando varios vinos, asegúrate de tomarte el tiempo necesario para apreciar y evaluar cada uno sin apresurarte.
9.	Comparación y reflexión:
•	Si estás catando varios vinos, compara tus notas y puntuaciones para determinar cuál te gusta más y por qué.
10.	Disfruta y aprende:
•	La cata de vinos es una experiencia placentera y educativa. Disfruta del proceso y sigue practicando para mejorar tu capacidad de apreciación de vinos.
Recuerda que la cata de vinos es subjetiva, y cada persona puede tener diferentes preferencias y percepciones. Lo más importante es disfrutar del vino y aprender a apreciar sus cualidades únicas.

CARACTERÍSTICAS DEL VINO
VINO: Cabernet Sauvignon 2019 del Valle de Guadalupe, Baja California, Mexico de la vinícola L.A. CETTO.
Uva: este vino tiene una uva originaria de Burdeos Francia adaptada al suelo de Valle de Guadalupe
Color: Es un vino tinto de color rojo rubí brillante.
Aroma: aromas a frutos rojos maduras y especias
Descripción: De tatino firme, intento y equilibrado.

Escenario: Ahora, tú serás un sommelier llamado Salomón y estarás con un novato de los vinos en la cual lo guiarás paso a paso por todo el procedimiento de la cata de vino para que pueda descubrir y degustar del vino descrito anteriormente. 
Personalidad del sommelier: El sommelier es una persona muy conocedora y divertida para guiar en la degustación. 

Recuerda: El sommelier no debe de darle la respuesta a la persona directamente, si no que debe guiarlo para que la otra persona lo descubra por su cuenta. Además, intenta expandir sobre los comentarios que hace la persona, por ejemplo, si la persona solo dice que ve un color rojo, 
después el sommelier debe preguntar ¿Qué tipo de rojo? Y darle ejemplos para llegar al color indicar. De la misma forma al hablar sobre los aromas, se busca que lo guie para que logre encontrar los aromas primarios, secundarios y si es posible terciarios del vino.  

además, recuerda siempre mantener la conversación relacionada con el vino, si alguien te habla de otra cosa el sommelier debe regresar al tema del vino y continuar en dónde se quedó del proceso de la cata. 

Ahora, escribe la primera línea de lo que diría el sommelier al invitado. Recuerda que tú serás el sommelier y siempre respondes en máximo un párrafo. Sommelier:


"""


# Global flag to control the loading sound playback
stop_loading_sound = False

def play_loading_sound_continuously():
    global stop_loading_sound
    while not stop_loading_sound:
        play(loading_sound)
        time.sleep(0.05)  # Adjust the sleep time as needed


# Define a global variable to store the response text
respuesta = ""
current_audio = None
current_audio_path = None
audio_output = widgets.Output()
display(audio_output)

# Function to generate speech for the response
def generate_speech(response_text):
    global current_audio
    global current_audio_path
    global stop_loading_sound

    palabras = len(response_text.split())
    espera = (palabras * 60) / 150 + 2

    # Define the path for the output file
    speech_file_path = Path("speech.mp3")

    # Call the OpenAI API to generate speech
    response = openai.audio.speech.create(
        model="tts-1",
        voice="alloy",
        input=response_text
    )


    # Stream the response to the specified file
    response.stream_to_file(speech_file_path)
    current_audio_path = str(speech_file_path)
    
    stop_loading_sound = True

    # Clear the previous audio from the audio output widget
    with audio_output:
        clear_output(wait=True)
        display(Audio(str(speech_file_path), autoplay=True))
    
    
    
    
respuesta = conversation.predict(input=prompt)

generate_speech(respuesta)

# ... (Rest of your code remains the same)
messages = Queue()
recordings = Queue()


record_button = widgets.Button(
    description="Record",
    disabled=False,
    button_style="success",
    icon="microphone"
)

stop_button = widgets.Button(
    description="Stop",
    disabled=False,
    button_style="warning",
    icon="stop"
)

output = widgets.Output()

def start_recording(data):
    global MENSAJE
    MENSAJE = ""
    messages.put(True)
    
    with output:
        display("Starting...")
        record = Thread(target=record_microphone)
        record.start()
        
        transcribe = Thread(target=speech_recognition, args=(output,))
        transcribe.start()


# Function to handle the user's speech input
def handle_user_input(user_input):
    global respuesta
    
    user_input = user_input + ". Ahora, dame la siguiente línea del sommelier en solo un párrafo. Sommelier:"

    respuesta = conversation.predict(input=user_input)
    generate_speech(respuesta)

# ... (Rest of your code remains the same)

# Function to handle the "Stop" button click
def stop_recording(data):
    global current_audio_path
    global stop_loading_sound

    with output:
        messages.get()
        display("Stopped.")
        # Play the loading sound
        #play(loading_sound)
        
        # Start playing the loading sound in a separate thread
        stop_loading_sound = False
        loading_thread = threading.Thread(target=play_loading_sound_continuously)
        loading_thread.start()

        #display("Transcribed Text:", MENSAJE)

        user_input = MENSAJE

        # Handle the user's input and generate a response
        handle_user_input(user_input)
        
        

        
record_button.on_click(start_recording)
stop_button.on_click(stop_recording)

display(record_button, stop_button, output)
